<a href="https://colab.research.google.com/github/chitraju-chaithanya/personalAIAgent/blob/main/HomeMatch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Personalized AI Agent


In [1]:
!pip install langchain==0.0.305
!pip install openai==0.28.1
!pip install pydantic>=1.10.12
!pip install pytest>=7.4.0
!pip install sentence-transformers>=2.2.0
!pip install transformers>=4.31.0
!pip install jupyter==1.0.0
!pip install chromadb==0.4.15
!pip install tiktoken

Provide api_key

In [2]:
import os
os.environ["OPENAI_API_KEY"] ="OPENAI_API_KEY"


Using a Large Language Model (LLM) to generate at least 10 diverse and realistic real estate listings containing facts about the real estate.

Results are then saved to Home.csv file.

In [3]:
import openai

prompt = """
Create a unique 10 listing in a csv format with Neighborhood,Price,Bedrooms,Bathrooms,House Size,Description,Neighborhood Description as columns.


Example:
Neighborhood,Price,Bedrooms,Bathrooms,House Size,Description,Neighborhood Description
Green Oaks,"$800,000",3,2,"2,000 sqft","Welcome to this eco-friendly oasis nestled in the heart of Green Oaks. This charming 3-bedroom", "2-bathroom home boasts energy-efficient features such as solar panels and a well-insulated structure".



Must have more than 10 unique listings in csv format is needed with unique Neighborhood,Price,Bedrooms,Bathrooms,House Size,Description,Neighborhood Description as columns.
"""

messages = [{"role": "system", "content": prompt}]

response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=messages,
    temperature=0,
    max_tokens=1000,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
)

print(response.choices[0].message.content)

save_file = open("Home.csv", "w")
save_file.write(response.choices[0].message.content)
save_file.close()

Neighborhood,Price,Bedrooms,Bathrooms,House Size,Description,Neighborhood Description
Willow Creek,"$650,000",4,3,"2,500 sqft","Beautiful 4-bedroom home in the serene Willow Creek neighborhood. This spacious house features a large backyard and modern finishes throughout.","Willow Creek is known for its family-friendly atmosphere and top-rated schools."
Pine Ridge,"$900,000",5,4,"3,000 sqft","Luxurious 5-bedroom estate in Pine Ridge with stunning mountain views. This home boasts high-end finishes, a gourmet kitchen, and a private pool.","Pine Ridge offers a peaceful retreat from the city with plenty of outdoor activities nearby."
Sunset Hills,"$750,000",3,2,"1,800 sqft","Charming 3-bedroom cottage in the desirable Sunset Hills neighborhood. This home features a cozy fireplace, hardwood floors, and a beautifully landscaped yard.","Sunset Hills is known for its tree-lined streets, friendly neighbors, and close proximity to downtown."
Riverbend,"$1,200,000",6,5,"4,500 sqft","Magnificent 6-

Display data from Home.csv file.

In [4]:
import pandas as pd
#Display 10 unique listing
df = pd.read_csv("./Home.csv")
print(df)


        Neighborhood       Price  Bedrooms  Bathrooms  House Size  \
0       Willow Creek    $650,000         4          3  2,500 sqft   
1         Pine Ridge    $900,000         5          4  3,000 sqft   
2       Sunset Hills    $750,000         3          2  1,800 sqft   
3          Riverbend  $1,200,000         6          5  4,500 sqft   
4        Maple Grove    $500,000         3          2  1,500 sqft   
5    Oakwood Estates    $850,000         4          3  2,800 sqft   
6        Meadowbrook    $600,000         3          2  1,700 sqft   
7  Hillcrest Heights    $950,000         5          4  3,200 sqft   
8          Sunnyvale    $700,000         3          2  1,900 sqft   
9   Lakeview Terrace  $1,100,000         4          3  3,500 sqft   

                                         Description  \
0  Beautiful 4-bedroom home in the serene Willow ...   
1  Luxurious 5-bedroom estate in Pine Ridge with ...   
2  Charming 3-bedroom cottage in the desirable Su...   
3  Magnificent 6

Load data from csv file and persist in chroma vector database.

In [5]:
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain import LLMChain
from langchain.chains.question_answering import load_qa_chain
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings

# Configuration
CHROMA_PATH = "chroma"
CHROMA_SETTINGS = {"name": "home_listings", "path": CHROMA_PATH}

#load data from csv
loader = CSVLoader(file_path='./Home.csv')
docs = loader.load()


splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
split_docs = splitter.split_documents(docs)

embeddings = OpenAIEmbeddings()

#save data to chroma vector db.
db = Chroma.from_documents(split_docs, embeddings)




Logic for Searching and Augmenting Listing Descriptions using LLM.

In [6]:

from langchain.chat_models import ChatOpenAI

query = """
A comfortable three-bedroom house with a spacious kitchen and a cozy living room.
A quiet neighborhood, good local schools, and convenient shopping options.
A backyard for gardening, a two-car garage, and a modern, energy-efficient heating system.
Easy access to a reliable bus line, proximity to a major highway, and bike-friendly roads.
A beautiful view of the mountains and a private deck for outdoor relaxation.
"""

print("-------------------")
print("Query: ", query)
print("-------------------")


print("Results from similarity search")
print("-------------------")

#Perform similarity search for top 3 results
#It also considers for personalized augmentation based on user preference.
context = db.similarity_search_with_relevance_scores(query, k=3, augmentation_kwargs={"personalize": True})

#context from similarity search
for doc, score in context:
  print(f"Document: {doc.page_content}")
  print(f"Score: {score}")
  print("-------------------")


prompt = PromptTemplate(
        template="{query}\nContext: {context}",
        input_variables=["query", "context"],
    )

model = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo")
summary = model.predict(prompt.format(query=query, context=context))

#Results from user preference
print("*****************************")
print("Personalized results from user preference")
print("*****************************")
print(summary)



-------------------
Query:  
A comfortable three-bedroom house with a spacious kitchen and a cozy living room.
A quiet neighborhood, good local schools, and convenient shopping options.
A backyard for gardening, a two-car garage, and a modern, energy-efficient heating system.
Easy access to a reliable bus line, proximity to a major highway, and bike-friendly roads.
A beautiful view of the mountains and a private deck for outdoor relaxation.

-------------------
Results from similarity search
-------------------
Document: Neighborhood: Meadowbrook
Price: $600,000
Bedrooms: 3
Bathrooms: 2
House Size: 1,700 sqft
Description: Inviting 3-bedroom ranch in the peaceful Meadowbrook neighborhood. This home offers an open floor plan, vaulted ceilings, and a covered patio for outdoor entertaining.
Neighborhood Description: Meadowbrook is a quiet community with walking trails, parks, and easy access to shopping and dining.
Score: 0.7840470082223654
-------------------
Document: Neighborhood: Sunny

References:

1. https://github.com/
2. https://chat.openai.com/
3. https://udacity.com/